# Uptake Reactions
This notebook adds uptake reactions for [phenol](https://www.genome.jp/dbget-bin/www_bget?C00146), [vanillate](https://www.genome.jp/dbget-bin/www_bget?C06672), [4-hydroxybenzoate](https://www.genome.jp/dbget-bin/www_bget?C00156), [guaiacol](https://www.genome.jp/dbget-bin/www_bget?C01502), and [benzoate](https://www.genome.jp/dbget-bin/www_bget?C00180) to the *R.opacus* genome scale model.

### The model was made using CarveMe and the commands:
The .faa assembly file was downloaded from [NCBI](https://www.ncbi.nlm.nih.gov/assembly/GCF_020542785.1). 
<ol>
    <li>carve r_opacus_bologna.faa -u grampos -o r_opacus_bologna_raw.xml</li>
    <li>gapfill r_opacus_bologna.xml -m M9,LB -o r_opacus_bologna_gapfilled.xml</li>
</ol>


###  Method
<ol>
    <li>Add metabolites neccessary for aromatic growth</li>
    <li>Add reactions to connect the metabolites to the metabolic network</li>
</ol>

###  Required Additional Metabolites
<ul>
    <li>Phenol: extracellular phenol and cytosolic phenol</li>
    <li>Vanillate: extracellular vanillate</li>
    <li>4-Hydroxybenzoate: none</li>
    <li>Guaiacol: extracellular guaiacol and cytosolic guaiacol</li>
    <li>Benzoate: none</li>
</ul>

###  Required Additional Reactions
<ul>
    <li>Phenol: phenol exchange, phenol transport, and phenol monooxygenase</li>
    <li>Vanillate: vanillate exchange, vanillate transport</li>
    <li>4-Hydroxybenzoate: none</li>
    <li>Guaiacol: guaiacol exchange and guaiacol transport, and guaicaol O-demethylase</li>
    <li>Benzoate: none</li>
</ul>

In [1]:
import cobra

### Load genome-scale model
Get *R.opacus* NCBI model generated by CarveMe

In [2]:
model = cobra.io.read_sbml_model("../GSMs/r_opacus_bologna_gapfilled.xml")
model

Name,italian_3
Memory address,0x07f01594953d0
Number of metabolites,1947
Number of reactions,3011
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


### Add neccessary metabolites to the model

In [3]:
# Extracellular phenol 
phenol_e = cobra.Metabolite(
    'phenol_e',
    formula='C6H6O',
    name='Phenol',
    compartment='C_e')

# Cytosolic phenol
phenol_c = cobra.Metabolite(
    'phenol_c',
    formula='C6H6O',
    name='Phenol',
    compartment='C_c')

# Extracellular 4-hydroxybenzoate
hbz_e = cobra.Metabolite(
    '4hbz_e',
    formula='C7H5O3',
    name='4-Hydroxybenzoate',
    compartment='C_e')

# Extracellular guaiacol
guaiacol_e = cobra.Metabolite(
    'guaiacol_e',
    formula='C7H8O2',
    name='Guaiacol',
    compartment='C_e')

# Cytosolic guaiacol
guaiacol_c = cobra.Metabolite(
    'guaiacol_c',
    formula='C7H8O2',
    name='Guaiacol',
    compartment='C_c')

# Add a metabolites for uptake pathway of vanillate and hydroxybenzoate

# 4-Carboxymuconolactone
cml_4 = cobra.Metabolite(
    '4cml_c',
    formula='C7H4O6',
    name='4-Carboxymuconolactone',
    compartment='C_c')

Add reactions for phenol exchange, transport, and [conversion to catechol](https://www.kegg.jp/entry/R10043) to the model

In [4]:
phenol_exchange = cobra.Reaction('EX_phenol_e')
phenol_exchange.name = 'Phenol exchange'
phenol_exchange.lower_bound = 0.  
phenol_exchange.upper_bound = 1000.  
phenol_exchange.add_metabolites({
    phenol_e: -1.0,
})

phenol_transport = cobra.Reaction('phenol_transport')
phenol_transport.name = 'Phenol transport'
phenol_transport.lower_bound = 0. 
phenol_transport.upper_bound = 1000. 
phenol_transport.add_metabolites({
    phenol_e: -1.0,
    phenol_c:  1.0,
}) 

# EC 1.14.13.244
phenol_monooxygenase = cobra.Reaction('PHEMOX')
phenol_monooxygenase.name = 'Phenol monooxygenase'
phenol_monooxygenase.lower_bound = 0.
phenol_monooxygenase.upper_bound = 1000.  # This is the default
phenol_monooxygenase.add_metabolites({
    phenol_c: -1.0,
    model.metabolites.get_by_id('nadh_c'):   -1.0,
    model.metabolites.get_by_id('h_c'): -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('nad_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
}) 


model.add_reactions([phenol_exchange])
model.add_reactions([phenol_transport])
model.add_reactions([phenol_monooxygenase])

Add guaiacol reactions for exchange, transport, and [conversion to catechol](https://www.genome.jp/entry/R12301) 

In [5]:
guaiacol_exchange = cobra.Reaction('EX_guaiacol_e')
guaiacol_exchange.name = 'Guaiacol exchange'
guaiacol_exchange.lower_bound = -1000.  
guaiacol_exchange.upper_bound = 1000.  
guaiacol_exchange.add_metabolites({
    guaiacol_e: -1.0,
})

guaiacol_transport = cobra.Reaction('guaiacol_transport')
guaiacol_transport.name = 'Guaiacol transport'
guaiacol_transport.lower_bound = -1000. 
guaiacol_transport.upper_bound = 1000. 
guaiacol_transport.add_metabolites({
    guaiacol_e: -1.0,
    guaiacol_c:  1.0,
}) 

# EC 1.14.14.-
guaiacol_o_demethylase = cobra.Reaction('GUADEM')
guaiacol_o_demethylase.name = 'Guaicaol O-demethylase'
guaiacol_o_demethylase.lower_bound = 0.
guaiacol_o_demethylase.upper_bound = 1000.  # This is the default
guaiacol_o_demethylase.add_metabolites({
    guaiacol_c: -1.0,
    model.metabolites.get_by_id('nadph_c'):   -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('fald_c'):  1.0,
    model.metabolites.get_by_id('nadp_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
    
}) 

model.add_reactions([guaiacol_exchange])
model.add_reactions([guaiacol_transport])
model.add_reactions([guaiacol_o_demethylase])

Ensure that the TCA cycle is complete

In [6]:
model.reactions.get_by_id('SUCD1').upper_bound = 1000
model.reactions.get_by_id('SUCD1')

Reaction identifier,SUCD1
Name,Succinate dehydrogenase
Memory address,0x07f02064d6c10
Stoichiometry,fad_c + succ_c <=> fadh2_c + fum_c Flavin adenine dinucleotide oxidized + Succinate <=> Flavin adenine dinucleotide reduced + Fumarate
GPR,WP_005240907_1 or ( WP_005247561_1 and WP_005247564_1 and WP_005247566_1 ) or ( WP_005239878_1 an...
Lower bound,-1000.0
Upper bound,1000


Add reaction to let hydrogen out of the cell

In [7]:
# hyrdogen_transport = cobra.Reaction('H2td')
# hyrdogen_transport.name = 'Hydrogen Transport'
# hyrdogen_transport.lower_bound = -1000.
# hyrdogen_transport.upper_bound = 1000.  # This is the default
# hyrdogen_transport.add_metabolites({
#     model.metabolites.get_by_id('h2_c'): -1.0,
#     # may need to add h2_e
# }) 

# model.add_reactions([hyrdogen_transport])

#### Define minimal media where the only carbon containing inputs are carbon sources

In [8]:
model.medium = {
 'EX_glc__D_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
#  'EX_phenol_e': 1000.0,
#  'EX_vanlt_e': 1000.0,
#  'EX_guaiacol_e': 1000.0,
#  'EX_tag': 1000.0
}

Set some bounds to prevent known futile cycles

In [9]:
for r_id in ['HADPCOADH3']:
    model.reactions.get_by_id(r_id).lower_bound = 0
    model.reactions.get_by_id(r_id).upper_bound = 0

In [10]:
# for r_id in ['QMO2', 'HYD1pp', 'MALS', 'GLYAT']:
#     model.reactions.get_by_id(r_id).lower_bound = 0
#     model.reactions.get_by_id(r_id).upper_bound = 0

Confirm the model can grow with glucose

In [11]:
glucose_solution = []
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    glucose_solution = model.optimize()
    growth_flux = glucose_solution.fluxes['Growth']
    print(f'10 mmol of glucose can make {growth_flux:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
10.0 EX_glc__D_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10 mmol of phenol can make 0.440 g of biomass


In [12]:
# glucose_solution.fluxes['H2td']

In [13]:
# for rxn, flux in glucose_solution.fluxes.items():
#     if abs(flux) > 0.001:
#         print(rxn, flux)

In [14]:
phenol_solution = []
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 10.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
#     phenol_solution = model.optimize()
    phenol_solution = cobra.flux_analysis.pfba(model)
    growth_flux = phenol_solution.fluxes['Growth']
    print(f'10 mmol of phenol can make {growth_flux:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10.0 EX_phenol_e
10 mmol of phenol can make 0.227 g of biomass


In [15]:
# for r_id, flux in phenol_solution.fluxes.items():
#     if abs(flux > 0.001):
#         print(r_id, round(flux,3))

In [16]:
# Track flow of carbon in phenol_solution

In [17]:
def producing_reactions(solution, met_id):
    # create object to hold producting reactions
    producing_rxns = {}

    # find the reactions that produce the metabolite
    for rxn_id, flux in solution:
        rxn = model.reactions.get_by_id(rxn_id)
        reactants = [m.id for m in rxn.reactants]
        products = [m.id for m in rxn.products]
        
        if (flux < 0 and met_id in reactants) or (flux > 0 and met_id in products):
            producing_rxns[rxn_id] = abs(flux)
            
    return producing_rxns

def consuming_reactions(solution, met_id):
    # create object to hold producting reactions
    consuming_rxns = {}

    # find the reactions that produce the metabolite
    for rxn_id, flux in solution:
        rxn = model.reactions.get_by_id(rxn_id)
        reactants = [m.id for m in rxn.reactants]
        products = [m.id for m in rxn.products]
        
        if (flux > 0 and met_id in reactants) or (flux < 0 and met_id in products):
            consuming_rxns[rxn_id] = abs(flux)
            
    return consuming_rxns

def fluxes_for_metabolite(solution, met_id):
    fluxes = {}
    fluxes['met_id'] = met_id
    fluxes['producing'] = producing_reactions(solution, met_id)
    fluxes['consuming'] = consuming_reactions(solution, met_id)
    return fluxes

def print_fluxes_for_metabolite(met_fluxes):
    met_id = met_fluxes['met_id']
    print(f'{met_id} -- {model.metabolites.get_by_id(met_id).name}')
    print('Producing reactions:')
    for r_id, flux in met_fluxes['producing'].items():
        rxn = model.reactions.get_by_id(r_id)
        print(f'{round(flux, 3)} {rxn} {rxn.name}')
    print('Consuming reactions:')
    for r_id, flux in met_fluxes['consuming'].items():
        rxn = model.reactions.get_by_id(r_id)
        print(f'{round(flux, 3)} {rxn} {rxn.name}')
    print()
#     print(met_fluxes['consuming'])
    

In [19]:
def carbon_flow_tracker(solution, metabolite_list):
    # filter out low flux reactions
    relevant_fluxes = [(rxn_id, flux) for rxn_id, flux in solution.fluxes.items() if abs(flux) > 0.001]
    

    for m in metabolite_list:
        met_fluxes = fluxes_for_metabolite(relevant_fluxes, m)
        print_fluxes_for_metabolite(met_fluxes)

    
carbon_flow_tracker(
    phenol_solution, 
    [
     'phenol_c', 'catechol_c', 'ccmuac_c', 'mucl_c', '5odhf2a_c', '3oxoadp_c', 'oxadpcoa_c', '3hadpcoa_c',
     'accoa_c', 'succoa_c',
     'h2_c', 'q8_c', 'q8h2_c', 'o2s_c', 'h2o2_c'
    ]

)

phenol_c -- Phenol
Producing reactions:
10.0 phenol_transport: phenol_e --> phenol_c Phenol transport
Consuming reactions:
10.0 PHEMOX: h_c + nadh_c + o2_c + phenol_c --> catechol_c + h2o_c + nad_c Phenol monooxygenase

catechol_c -- Catechol
Producing reactions:
10.0 PHEMOX: h_c + nadh_c + o2_c + phenol_c --> catechol_c + h2o_c + nad_c Phenol monooxygenase
Consuming reactions:
10.0 CATDOX: catechol_c + o2_c --> ccmuac_c + 2.0 h_c Catechol 1,2-dioxygenase;

ccmuac_c -- Cis,cis-Muconate
Producing reactions:
10.0 CATDOX: catechol_c + o2_c --> ccmuac_c + 2.0 h_c Catechol 1,2-dioxygenase;
Consuming reactions:
10.0 MUCCYCI: ccmuac_c + h_c <=> mucl_c Muconate cycloisomerase

mucl_c -- Mucolactone
Producing reactions:
10.0 MUCCYCI: ccmuac_c + h_c <=> mucl_c Muconate cycloisomerase
Consuming reactions:
10.0 MUCLI: mucl_c <=> 5odhf2a_c Mucolactone isomerase

5odhf2a_c -- 5-Oxo-4,5-dihydrofuran-2-acetate
Producing reactions:
10.0 MUCLI: mucl_c <=> 5odhf2a_c Mucolactone isomerase
Consuming reacti

In [25]:
for r in model.metabolites.get_by_id('cit_c').reactions:
    print(r)

CITtupp: cit_p --> cit_c
ACONT: cit_c <=> icit_c
CITACt: ac_c + cit_e <-- ac_e + cit_c
FEDCabc: atp_c + 2.0 cit_e + fe3_e + h2o_c --> adp_c + 2.0 cit_c + fe3_c + h_c + pi_c
FE3DCITR5: 2.0 fe3dcit_c + nadph_c --> 4.0 cit_c + 2.0 fe2_c + h_c + nadp_c
FE3DCITCH: 2.0 cit_c + fe3_c <=> fe3dcit_c
ACONTa: cit_c <=> acon_C_c + h2o_c
FE3DCITabcpp: atp_c + fe3dcit_p + h2o_c --> adp_c + 2.0 cit_c + fe3_c + h_c + pi_c
FE3DCITabc: atp_c + fe3dcit_e + h2o_c --> adp_c + 2.0 cit_c + fe3_c + h_c + pi_c
CITL: cit_c --> ac_c + oaa_c
CITt14: ca2_e + cit_e + h_e --> ca2_c + cit_c + h_c
CS: accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
CITt4pp_1: cit_p + na1_p --> cit_c + na1_c
CITt_kt: cit_p + h_p --> cit_c + h_c


In [26]:
for r in model.metabolites.get_by_id('icit_c').reactions:
    print(r)

ACONT: cit_c <=> icit_c
ICL: icit_c --> glx_c + succ_c
ICITt4pp: icit_p + na1_p --> icit_c + na1_c
ACONTb: acon_C_c + h2o_c <=> icit_c
ICDHyr: icit_c + nadp_c <=> akg_c + co2_c + nadph_c
ICDHx: icit_c + nad_c <=> akg_c + co2_c + nadh_c
ICITt_kt: h_p + icit_p --> h_c + icit_c
ICITtupp: icit_p --> icit_c


In [27]:
for r in model.metabolites.get_by_id('akg_c').reactions:
    print(r)

GLUSy: akg_c + gln__L_c + h_c + nadph_c --> 2.0 glu__L_c + nadp_c
CYSTA: akg_c + cys__L_c --> glu__L_c + mercppyr_c
ILETA2: 3mop_c + glu__L_c <=> akg_c + ile__L_c
OXGDC: akg_c + h_c --> co2_c + sucsal_c
AKGDa: akg_c + h_c + lpam_c <=> co2_c + sdhlam_c
GLUSx: akg_c + gln__L_c + h_c + nadh_c --> 2.0 glu__L_c + nad_c
EHGLAT: akg_c + e4hglu_c --> 4h2oglt_c + glu__L_c
3SALATAi: 3sala_c + akg_c + h_c <=> 3snpyr_c + glu__L_c
GLUTARH: akg_c + glutar_c + o2_c --> S2hglut_c + co2_c + succ_c
HSTPT: glu__L_c + imacp_c --> akg_c + hisp_c
ILETA: akg_c + ile__L_c <=> 3mop_c + glu__L_c
ORNTA_1: akg_c + orn__L_c --> glu5sa_c + glu__L_c
KGD2: co2_c + 2.0 fdxrd_c + h_c + succoa_c <-- akg_c + coa_c + 2.0 fdxox_c
DOPDH: 25dop_c + h2o_c + nadp_c <=> akg_c + 2.0 h_c + nadph_c
LEUTA: akg_c + leu__L_c <=> 4mop_c + glu__L_c
TYRTAi: 34hpp_c + glu__L_c --> akg_c + tyr__L_c
ABTA: 4abut_c + akg_c --> glu__L_c + sucsal_c
PSERT: 3php_c + glu__L_c --> akg_c + pser__L_c
OHPBAT: glu__L_c + ohpb_c <=> akg_c + phthr_c
PTR

In [28]:
for r in model.metabolites.get_by_id('succoa_c').reactions:
    print(r)

3OADPCOAT: 3oxoadp_c + succoa_c --> oxadpcoa_c + succ_c
AKGDb: coa_c + sdhlam_c --> dhlam_c + succoa_c
AKGDH: akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
SSCOARy: h_c + nadph_c + succoa_c <=> coa_c + nadp_c + sucsal_c
OCOAT4: dhcrn_c + succoa_c --> dhcrncoa_c + succ_c
SUCBZT1: bz_c + 4.0 h_c + succoa_c <=> benzcoa_c + succ_c
OCOAT1: acac_c + succoa_c <=> aacoa_c + succ_c
KGD2: co2_c + 2.0 fdxrd_c + h_c + succoa_c <-- akg_c + coa_c + 2.0 fdxox_c
MMM2: succoa_c --> mmcoa__R_c
SUCBZT2: 4hbz_c + succoa_c <=> 4hbzcoa_c + succ_c
THDPS: h2o_c + succoa_c + thdp_c --> coa_c + sl2a6o_c
BSCT: coa_c + 4.0 h_c + sbzcoa_c <=> benzcoa_c + succoa_c
ALAS: gly_c + h_c + succoa_c <=> 5aop_c + co2_c + coa_c
3OXCOAT: coa_c + oxadpcoa_c --> accoa_c + succoa_c
BZSCT: bzsucc_c + succoa_c <=> bzsuccoa_c + succ_c
SUCOAS: atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c


In [29]:
for r in model.metabolites.get_by_id('succ_c').reactions:
    print(r)

3OADPCOAT: 3oxoadp_c + succoa_c --> oxadpcoa_c + succ_c
SSALy: h2o_c + nadp_c + sucsal_c --> 2.0 h_c + nadph_c + succ_c
FRD3: 2dmmql8_c + fum_c --> 2dmmq8_c + succ_c
HMR_9610: na1_e + succ_e --> na1_c + succ_c
ICL: icit_c --> glx_c + succ_c
TARTRt7: succ_c + tartr__L_e <=> succ_e + tartr__L_c
MCITL2: micit_c <=> pyr_c + succ_c
SHSL2r: h2s_c + suchms_c <=> h_c + hcys__L_c + succ_c
SSALx: h2o_c + nad_c + sucsal_c --> 2.0 h_c + nadh_c + succ_c
ASPO5: asp__L_c + fum_c --> h_c + iasp_c + succ_c
OCOAT4: dhcrn_c + succoa_c --> dhcrncoa_c + succ_c
FRD: fadh2_c + fum_c --> fad_c + succ_c
SHSL4r: h2o_c + suchms_c --> 2obut_c + h_c + nh4_c + succ_c
GLUTARH: akg_c + glutar_c + o2_c --> S2hglut_c + co2_c + succ_c
OCOAT1: acac_c + succoa_c <=> aacoa_c + succ_c
SUCBZT1: bz_c + 4.0 h_c + succoa_c <=> benzcoa_c + succ_c
TAUDO: akg_c + o2_c + taur_c --> aacald_c + co2_c + h_c + so3_c + succ_c
SUCBZT2: 4hbz_c + succoa_c <=> 4hbzcoa_c + succ_c
FRD2: fum_c + mql8_c --> mqn8_c + succ_c
HKNDDH: h2o_c + hkndd

In [33]:
model.reactions.get_by_id('SUCD1')

Reaction identifier,SUCD1
Name,Succinate dehydrogenase
Memory address,0x07f17c8471bd0
Stoichiometry,fad_c + succ_c <-- fadh2_c + fum_c Flavin adenine dinucleotide oxidized + Succinate <-- Flavin adenine dinucleotide reduced + Fumarate
GPR,WP_005240907_1 or ( WP_005247561_1 and WP_005247564_1 and WP_005247566_1 ) or ( WP_005239878_1 an...
Lower bound,-1000.0
Upper bound,0.0


In [31]:
for r in model.metabolites.get_by_id('fum_c').reactions:
    print(r)

FUMt2_2: fum_e + 2.0 h_e --> fum_c + 2.0 h_c
ARGSL: argsuc_c <=> arg__L_c + fum_c
BZSS: fum_c + tol_c <=> bzsucc_c
FRD3: 2dmmql8_c + fum_c --> 2dmmq8_c + succ_c
FUMt2r: fum_e + h_e --> fum_c + h_c
ASPT: asp__L_c --> fum_c + nh4_c
FUM: fum_c + h2o_c <=> mal__L_c
MFUMt8: fum_e + na1_e --> fum_c + na1_c
ADSL2r: 25aics_c <=> aicar_c + fum_c
ASPO5: asp__L_c + fum_c --> h_c + iasp_c + succ_c
FRD: fadh2_c + fum_c --> fad_c + succ_c
FUMt1pp: fum_p <=> fum_c
HKNTDH: h2o_c + hkntd_c --> fum_c + h_c + op4en_c
FUMt2_2pp: fum_p + 2.0 h_p --> fum_c + 2.0 h_c
FUMAC: 4fumacac_c + h2o_c --> acac_c + fum_c + h_c
MALEI: male_c <=> fum_c
FRD2: fum_c + mql8_c --> mqn8_c + succ_c
ADSL1r: dcamp_c <=> amp_c + fum_c
FRDx: fum_c + h_c + nadh_c --> nad_c + succ_c
SUCD1: fad_c + succ_c <-- fadh2_c + fum_c


In [32]:
for r in model.metabolites.get_by_id('mal__L_c').reactions:
    print(r)

MDH3_1: mal__L_c + mqn7_c --> mql7_c + oaa_c
DMALRED: fad_c + mal__L_c --> fadh2_c + oaa_c
ME2: mal__L_c + nadp_c --> co2_c + nadph_c + pyr_c
FUM: fum_c + h2o_c <=> mal__L_c
ME1: mal__L_c + nad_c --> co2_c + nadh_c + pyr_c
MALt2_2: 2.0 h_e + mal__L_e --> 2.0 h_c + mal__L_c
MALt2r: h_e + mal__L_e --> h_c + mal__L_c
MALt4: mal__L_e + na1_e --> mal__L_c + na1_c
MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
MALS: accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c
MDH2: mal__L_c + q8_c --> oaa_c + q8h2_c
MDH3: mal__L_c + mqn8_c --> mql8_c + oaa_c
MALt2_2pp: 2.0 h_p + mal__L_p --> 2.0 h_c + mal__L_c


In [19]:
model.metabolites.get_by_id('2aobut_c')

Metabolite identifier,2aobut_c
Name,L-2-Amino-3-oxobutanoate
Memory address,0x07f17ca78f610
Formula,C4H7NO3
Compartment,C_c
In 3 reaction(s),"GLYAT, THRD, AOBUTDs"


In [18]:
for r in model.metabolites.get_by_id('accoa_c').reactions:
    print(r)

A4HBCT: accoa_c + h4but_c --> 4hbutcoa_c + ac_c + h_c
ACACT2r: accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
PDH: coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
MACCOAT: 2maacoa_c + coa_c --> accoa_c + ppcoa_c
A24T: a24op_c + coa_c --> accoa_c + ala__D_c
ACACT7r: accoa_c + tdcoa_c <=> 3ohdcoa_c + coa_c
G1PACT: accoa_c + gam1p_c --> acgam1p_c + coa_c + h_c
KAS13: accoa_c + 23.0 h_c + 8.0 malcoa_c + 16.0 nadph_c --> 8.0 co2_c + 9.0 coa_c + 7.0 h2o_c + 16.0 nadp_c + ocdca_c
ALGAC4: 4.0 accoa_c + prealginate__M_p --> 4.0 coa_c + prealgac_M4_p
AACLAM: acadl_c + coa_c <=> accoa_c + amp_c
3OXCOAT: coa_c + oxadpcoa_c --> accoa_c + succoa_c
ACS: ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c
FAO3: atp_c + 9.0 coa_c + 8.0 fad_c + 8.0 h2o_c + 8.0 nad_c + ocdca_c --> 9.0 accoa_c + amp_c + 8.0 fadh2_c + 8.0 h_c + 8.0 nadh_c + ppi_c
HIAA: h2o_c + hipocoa_c --> accoa_c + hia_c
ACTD2: actn__R_c + coa_c + nad_c --> acald_c + accoa_c + h_c + nadh_c
KAT30: 3ohd710coa_c + coa_c --> accoa_c + td_5_8_coa_c
K

In [17]:
for r in model.metabolites.get_by_id('q8h2_c').reactions:
    print(r)

LDH_D2: lac__D_c + q8_c --> pyr_c + q8h2_c
L_LACD2: lac__L_c + q8_c --> pyr_c + q8h2_c
G3PD5: glyc3p_c + q8_c --> dhap_c + q8h2_c
GLYCTO2: glyclt_c + q8_c --> glx_c + q8h2_c
POX: h2o_c + pyr_c + q8_c --> ac_c + co2_c + q8h2_c
TETDHpp3: q8h2_c + tet_p --> 2.0 h_p + q8_c + 2.0 tsul_p
QMO2: 2.0 o2_c + q8h2_c --> 2.0 h_c + 2.0 o2s_c + q8_c
TTONR2pp: q8h2_c + tton_e --> 4.0 h_c + q8_c + so3_e + tsul_e
MDH2: mal__L_c + q8_c --> oaa_c + q8h2_c
DHORD2: dhor__S_c + q8_c --> orot_c + q8h2_c
HYD1pp: h2_c + 2.0 h_c + q8_c --> 2.0 h_p + q8h2_c
GLCDpp: glc__D_p + h2o_p + q8_c --> glcn_p + h_p + q8h2_c
GAD2ktpp: glcn_p + q8_c <=> 2dhglcn_p + q8h2_c
NADH5: h_c + nadh_c + q8_c --> nad_c + q8h2_c
NADPHQR2: h_c + nadph_c + q8_c --> nadp_c + q8h2_c
CYTBD: 2.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 2.0 h_e + q8_c


In [18]:
for r in model.metabolites.get_by_id('q8_c').reactions:
    print(r)

LDH_D2: lac__D_c + q8_c --> pyr_c + q8h2_c
L_LACD2: lac__L_c + q8_c --> pyr_c + q8h2_c
G3PD5: glyc3p_c + q8_c --> dhap_c + q8h2_c
GLYCTO2: glyclt_c + q8_c --> glx_c + q8h2_c
POX: h2o_c + pyr_c + q8_c --> ac_c + co2_c + q8h2_c
TETDHpp3: q8h2_c + tet_p --> 2.0 h_p + q8_c + 2.0 tsul_p
QMO2: 2.0 o2_c + q8h2_c --> 2.0 h_c + 2.0 o2s_c + q8_c
TTONR2pp: q8h2_c + tton_e --> 4.0 h_c + q8_c + so3_e + tsul_e
MDH2: mal__L_c + q8_c --> oaa_c + q8h2_c
DHORD2: dhor__S_c + q8_c --> orot_c + q8h2_c
HYD1pp: h2_c + 2.0 h_c + q8_c --> 2.0 h_p + q8h2_c
GLCDpp: glc__D_p + h2o_p + q8_c --> glcn_p + h_p + q8h2_c
GAD2ktpp: glcn_p + q8_c <=> 2dhglcn_p + q8h2_c
NADH5: h_c + nadh_c + q8_c --> nad_c + q8h2_c
NADPHQR2: h_c + nadph_c + q8_c --> nadp_c + q8h2_c
CYTBD: 2.0 h_c + 0.5 o2_c + q8h2_c --> h2o_c + 2.0 h_e + q8_c


In [19]:
r1 = model.reactions.get_by_id('HADPCOADH')
r1

Reaction identifier,HADPCOADH
Name,3-hydroxyadipyl-CoA dehydrogenase
Memory address,0x07fd0cd0a0290
Stoichiometry,3hadpcoa_c --> h2_c + oxadpcoa_c (3S)-3-Hydroxyadipyl-CoA --> Hydrogen + 3-Oxoadipyl-CoA
GPR,WP_025432691_1 and WP_226955502_1
Lower bound,0.0
Upper bound,1000.0


In [166]:
r2 =model.reactions.get_by_id('HADPCOADH3')
r2

Reaction identifier,HADPCOADH3
Name,3-hydroxyadipyl-CoA dehydrogenase (NAD+)
Memory address,0x07f8aac77fe50
Stoichiometry,3hadpcoa_c + nad_c <=> h_c + nadh_c + oxadpcoa_c (3S)-3-Hydroxyadipyl-CoA + Nicotinamide adenine dinucleotide <=> H+ + Nicotinamide adenine dinucleotide - reduced + 3-Oxoadipyl-CoA
GPR,WP_025432691_1
Lower bound,-1000.0
Upper bound,1000.0


#### The model can grow if all nonboundary reactions are made reversible

#### Check that model can grow with each carbon source

In [15]:
# glucose
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of glucose can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
10.0 EX_glc__D_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10 mmol of glucose can make 0.684 g of biomass


In [16]:
# phenol
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 10.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of phenol can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10.0 EX_phenol_e
10 mmol of phenol can make 0.046 g of biomass


In [17]:
# vanillate
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 10.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of vanillate can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
10.0 EX_vanlt_e
1000.0 EX_zn2_e
10 mmol of vanillate can make 0.114 g of biomass


In [18]:
# 4-hydroxybenzoate
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 10.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of 4-hydroxybenzoate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_4hbz_e
1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10 mmol of 4-hydroxybenzoate can make 0.023 g of biomass


In [19]:
# guaiacol
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 10.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of guaiacol can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10.0 EX_guaiacol_e
10 mmol of guaiacol can make 0.137 g of biomass


In [20]:
# benzoate
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 10.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of benzoate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_bz_e
1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10 mmol of benzoate can make 0.091 g of biomass


#### Save model

In [26]:
model.id = 'ropacus_model_A'
model.name = 'R. opacus model A'
cobra.io.write_sbml_model(model, "../GSMs/ropacus_model_A.xml")
model

Name,ropacus_model_A
Memory address,0x07f66a61f33d0
Number of metabolites,1237
Number of reactions,1761
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"
